In [ ]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]
filename

Saving 서울시 주요 공원현황.csv to 서울시 주요 공원현황.csv


'서울시 주요 공원현황.csv'

In [ ]:
import numpy as np
import pandas as pd
import folium

In [125]:
df = pd.read_csv(filename, encoding='EUC-KR')

In [ ]:
df

In [126]:
area = df.면적.astype(str).apply(lambda x : x.strip('총 ').split()[0])

In [127]:
area.to_frame()

,면적
0,2896887㎡
1,80683㎡
2,9132690m²
3,480994㎡
4,2284085㎡
...,...
127,109635㎡
128,140022㎡
129,211392㎡
130,157535㎡


In [128]:
area[(area == 'nan') | (area == '휴양')]

6       휴양
109    nan
119    nan
131    nan
Name: 면적, dtype: object

In [129]:
area[(area == 'nan') | (area == '휴양')] = '0㎡'
area[-5:]

127    109635㎡
128    140022㎡
129    211392㎡
130    157535㎡
131         0㎡
Name: 면적, dtype: object

In [130]:
import re
area = area.apply(lambda x : float(re.sub('[㎡ m²㎥]', '', x)))

In [131]:
area = area.apply(lambda x : int(np.round(x)))

In [132]:
area

0      2896887
1        80683
2      9132690
3       480994
4      2284085
        ...   
127     109635
128     140022
129     211392
130     157535
131          0
Name: 면적, Length: 132, dtype: int64

2. 공원의 면적에 따라 분류 파생변수

In [133]:
bins = [-1, 100000, 1000000, 12000000]
label = ['소형', '중형', '대형']
size_info = [3, 7, 15]
scale = pd.cut(area, bins, labels=label)
size = pd.cut(area, bins, labels=size_info)

In [134]:
df['면적'] = area
df['분류'] = scale
df['크기'] = size

In [135]:
df = df[['공원명', '지역', 'X좌표(WGS84)', 'Y좌표(WGS84)', '면적', '분류', '크기']]
df.columns = ['공원명', '지역', '경도', '위도', '면적', '분류', '크기']
df.head()

,공원명,지역,경도,위도,면적,분류,크기
0,남산도시자연공원,중구,126.990377,37.550140,2896887,대형,15
1,길동생태공원,강동구,127.154779,37.540394,80683,소형,3
2,서울대공원,과천시,127.019846,37.426449,9132690,대형,15
3,서울숲,성동구,127.041798,37.543072,480994,중형,7
4,월드컵공원,마포구,126.878907,37.571805,2284085,대형,15


In [136]:
df.loc[129, '지역'] = '노원구'
df = df.drop(index=131)

In [137]:
df.to_csv('서울공원전처리.csv', index=False)

In [138]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11)

for i in df.index:
    folium.CircleMarker(
        radius=int(df.크기[i]),
        location = [df.위도[i], df.경도[i]], 
        # popup = folium.Popup(df.지역[i], max_width=200),
        tooltip = f'{df.공원명[i]}({df.면적[i]: d}㎡)', 
        color = 'red',
        fill=True,
        fill_color = 'crimson',
    ).add_to(map)
title = '<h3 align="center" style="font-size:20[x">서울시내 공원위치 크기</h3>'
map.get_root().html.add_child(folium.Element(title))
map